In [2]:
# Initial imports
import numpy as np
import pandas as pd 
from pandas import DataFrame, Series
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
%matplotlib inline

import random
import urllib.request
import requests
from bs4 import BeautifulSoup
import warnings
warnings.filterwarnings('ignore')

In [3]:
base_url = "https://www.superherodb.com/characters/"
list_chars_cols = ['Name', 'Url']

url = base_url 
source_code = requests.get(url)
plain_text = source_code.text
soup = BeautifulSoup(plain_text, "html.parser")

In [4]:
list_of_supers = soup.findAll('li', {'class': 'char-li'})

In [5]:
len(list_of_supers)

743

In [6]:
data = DataFrame(columns=list_chars_cols)
for l in list_of_supers:
    temp = DataFrame([[l.text, l.find('a').get('href')]])
    temp.columns = list_chars_cols
    data = data.append(temp, ignore_index=1)

In [7]:
data.head()

,Name,Url
0,3-D Man,/3-d-man/10-226/
1,A-Bomb,/a-bomb/10-10060/
2,Abe Sapien,/abe-sapien/10-956/
3,Abin Sur,/abin-sur/10-1460/
4,Abomination,/abomination/10-1/


In [8]:
char_base_url = 'https://www.superherodb.com'
stats_list = ['Url', 'Intelligence', 'Strength', 'Speed', 'Durability', 'Power', 
              'Combat', 'Full name', 'Alter Egos', 'Aliases', 'Place of birth', 
              'First appearance', 'Creator', 'Alignment', 'Gender', 'Race', 'Height', 
              'Weight', 'Eye color', 'Hair color', 'Occupation', 'Base', 'Team Affiliation', 'Relatives']
stat_data = DataFrame(columns=stats_list)
for index, row in data.iterrows():
    stat_map = {'Url': str(row['Url'])}
    char_url = char_base_url + str(row['Url'])
    source_code = requests.get(char_url)
    plain_text = source_code.text
    soup = BeautifulSoup(plain_text, "html.parser")
    attrs = soup.findAll('div', {'class': 'gridbarholder'})
    personal = soup.findAll('table', {'class': 'table'})
    for a in attrs[:6]:
        stat_name = a.find('div', {'class': 'gridbarlabel'}).text
        stat_value = a.find('div', {'class': 'gridbarvalue'}).text
        stat_map[str(stat_name)] = stat_value
    for p in personal[:3]:
        trs = p.findAll('tr')
        for tr in trs:
            stat_map[str(tr.find('td').text.strip())] = str(tr.findAll('td')[1].text.strip())
    stat_map[str(personal[3].findAll('td')[0].text.strip())] = str(personal[3].findAll('td')[1].text.strip())
    stat_map[str(personal[3].findAll('td')[2].text.strip())] = str(personal[3].findAll('td')[3].text.strip())
    for key in stat_map.keys():
        stat_data.loc[index,key] = stat_map[key]

In [72]:
full_data = pd.merge(data, stat_data, left_index=True, right_index=True)

In [73]:
full_data.drop('Url_x', 1,  inplace=True)
f = full_data.rename(index=str, columns={"Url_y": "Url"})

In [74]:
f[['Intelligence', 'Strength', 
   'Speed', 'Durability', 
   'Power', 'Combat']] = f[['Intelligence', 'Strength', 'Speed', 'Durability', 'Power', 'Combat']].astype(float)


In [75]:
f['Total Power'] = f['Intelligence'] +  f['Strength'] + f['Speed'] + f['Durability'] + f['Power'] + f['Combat']

In [76]:
f['Total Power'].fillna(0, inplace=True)

In [77]:
sorted_d = f.sort_values('Total Power', ascending=False)

In [81]:
sorted_d['Creator'][:10][0]

'DC Comics'

In [109]:
def get_char_data(char_name, dataset):
    d = False
    if not data.loc[data['Name'] == char_name.title()].empty:
        d = data.loc[data['Name'] == char_name.title()]
    return d

In [110]:
d =get_char_data('batman', f)

In [112]:
d.iloc[0]['Name']

'Batman'